# AE on proteins in SA representation - protein level

In [1]:
import glob
import os
from collections import Counter
import string
from keras import Input
from keras.layers import Dense, Lambda, Conv1D
import keras.backend as K
from keras.models import Model
from keras.objectives import binary_crossentropy, mse
import os
import random
import numpy as np
import NotebookLoader
from keras.optimizers import RMSprop, Adam
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
from tempfile import TemporaryFile
import csv

Using TensorFlow backend.


In [2]:
import Preprocessing as pre

importing Jupyter notebook from Preprocessing.ipynb


### Configs

In [3]:
lengths = {}
for f in pre.families:
    proteins = glob.glob(os.path.join(pre.family_paths[f], "*.out"))
    print("Proteins for family %s" %f)
    for p in proteins:
        print(p)
    lengths[f] = len(p)
total = sum([lengths[f] for f in pre.families])

Proteins for family fam_1
../dataset/families/fam_1/2EBN.lf_str.out
../dataset/families/fam_1/1JFX.lf_str.out
../dataset/families/fam_1/1ITX.lf_str.out
../dataset/families/fam_1/1NAR.lf_str.out
../dataset/families/fam_1/1KFW.lf_str.out
../dataset/families/fam_1/1B1Y.lf_str.out
../dataset/families/fam_1/1VFF.lf_str.out
../dataset/families/fam_1/1EDG.lf_str.out
../dataset/families/fam_1/1CNV.lf_str.out
Proteins for family fam_2
../dataset/families/fam_2/1HLB.lf_str.out
../dataset/families/fam_2/2LHB.lf_str.out
../dataset/families/fam_2/1DLW.lf_str.out
../dataset/families/fam_2/1ITH.lf_str.out
../dataset/families/fam_2/1ECA.lf_str.out
../dataset/families/fam_2/1ASH.lf_str.out
../dataset/families/fam_2/1MBA.lf_str.out
../dataset/families/fam_2/2HBG.lf_str.out
Proteins for family fam_3
../dataset/families/fam_3/2SAS.lf_str.out
../dataset/families/fam_3/1OMR.lf_str.out
../dataset/families/fam_3/1SRA.lf_str.out
../dataset/families/fam_3/1CB1.lf_str.out
../dataset/families/fam_3/1IQ3.lf_str.ou

In [4]:
padding = True
num_classes = 25
categorical = False
use_angles = True
max_length = 668
flatten = True

In [5]:
batch_size = 64
intermediate_dim = 25
epochs = 20
learning_rate = 0.001

### Build the autoencoder

In [6]:
def create_checkpoints(f):
    checkpoints_path = os.path.join("models_proteins", f)
    tensorboard_path = os.path.join("logs", f)
    cp_cb = ModelCheckpoint(filepath=os.path.join(checkpoints_path, "model_protein_level_" + f + ".hdf5"), monitor='val_loss',
                            save_best_only=True)
    tb_cb = TensorBoard(log_dir=tensorboard_path)
    return [cp_cb, tb_cb]

In [7]:
#autoencoder
def get_ae():
    if categorical:
        if not flatten:
            model_input = Input(shape=(None,num_classes))
        else:
            model_input = Input(shape=(max_length*num_classes,))
    elif use_angles:
        model_input = Input(shape=(max_length*3,))
    else:
        model_input = Input(shape=(max_length,))
    #x=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(model_input)
    #encoded=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1, name="encoded")(x)
    #x=Conv1D(num_classes, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(encoded)
    encoded= Dense(intermediate_dim, activation='sigmoid')(model_input)
    if categorical:
        if not flatten:
            x = Dense(num_classes, activation='sigmoid')(encoded)
        else:
            x = Dense(max_length*num_classes, activation='sigmoid')(encoded)
    elif use_angles:
        x = Dense(max_length*3, activation='sigmoid')(encoded)
    else:
        x = Dense(max_length, activation='sigmoid')(encoded)
    ae=Model(inputs=model_input, outputs=[x])
    opt=RMSprop(lr=learning_rate)
    ae.compile(optimizer=opt, loss='binary_crossentropy', metrics=['mean_absolute_error'])
    ae.summary()
    return ae

### Train and evaluate the autoencoder for specific classes

In [8]:
models = {"fam_1": "models_proteins/fam_1/model_protein_level_fam_1.hdf5",
          "fam_2": "models_proteins/fam_2/model_protein_level_fam_2.hdf5",
          "fam_3": "models_proteins/fam_3/model_protein_level_fam_3.hdf5",
          "fam_4": "models_proteins/fam_4/model_protein_level_fam_4.hdf5",
          "fam_5": "models_proteins/fam_5/model_protein_level_fam_5.hdf5",
          "fam_6": "models_proteins/fam_6/model_protein_level_fam_6.hdf5",
          "fam_7": "models_proteins/fam_7/model_protein_level_fam_7.hdf5",
          "fam_8": "models_proteins/fam_8/model_protein_level_fam_8.hdf5",
          "fam_9": "models_proteins/fam_9/model_protein_level_fam_9.hdf5"}

In [9]:
path = '/tmp/data_serialized_proteins_prot'
# for r in range(1, 10):
#     os.makedirs(os.path.join(path, 'fam_%d' % r))

In [10]:
def read_set_for_family(f, set_type):
    ds_path = os.path.join(path, f, set_type)
    files = glob.glob(os.path.join(ds_path, "*.npy"))
    new = np.load(files[0])
    for f in files[1:]:
        conf_f = np.load(f)
        new = np.concatenate([new,conf_f])
        del conf_f
    return new

In [11]:
def evaluate_for_fam(f):
    print("Test for autoencoder on fam %s" %f)
    train = read_set_for_family(f,"train")
    ae = load_model(models[f])
    ae.summary()
    losses_train = []
    for t in train:
        losses_train.append(ae.evaluate(np.array([t]),np.array([t]), verbose=0)[1])
    max_l = max(losses_train)
    print("Max loss is %f" %max_l)
    losses_test = []
    del train
    tp, tn, fp, fn = 0, 0, 0, 0
    tp_p, tn_p, fp_p, fn_p = 0, 0, 0, 0
    for ft in pre.families:
        print("Test for fam %s" %ft)
        ds_path = os.path.join(path, ft, "test")
        files = glob.glob(os.path.join(ds_path, "*.npy"))
        for file in files:
            test = np.load(file)
            total_nr = test.shape[0]
            total_loss = 0.0
            gt = 0
            ls = 0
            # check for each configuration the losses
            for t in test:
                loss=ae.evaluate(np.array([t]),np.array([t]), verbose=0)[1]
                total_loss +=loss
                if loss > max_l:
                    gt+=1
                else:
                    ls+=1
            if gt >= ls:
                # predict other family
                if ft == f:
                    fn+=1
                else:
                    tn+=1
            else:
                # predict current family
                if ft == f:
                    tp+=1 
                else:
                    fp+=1
            # compute the probability
            total_loss /= total_nr
            if total_loss > max_l:
                pr = 1 - max_l / (2 * total_loss)
            else:
                pr = total_loss / (2 * max_l)
            if pr >= 0.5:
                # predict other family
                if ft == f:
                    fn_p+=1
                else:
                    tn_p+=1
            else:
                # predict current family
                if ft == f:
                    tp_p+=1 
                else:
                    fp_p+=1                 
    return [tp, tn, fp, fn], [tp_p, tn_p, fp_p, fn_p]

In [12]:
res_fam = open('res_fam_conf_prot.csv', mode='w')
res_avg = open('res_avg_conf_prot.csv', mode='w')
writer_fam = csv.writer(res_fam, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_fam.writerow(['Iteration', 'Superfamily', 'TP', 'TN', 'FP', 'FN', 'Prec', 'Recall', 'Spec', 'AUC'])
writer_avg = csv.writer(res_avg, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_avg.writerow(['Iteration', 'Prec', 'Recall', 'Spec', 'AUC'])
for i in range(0,20):
    
    # serialize data
    for f in pre.families:
        proteins = glob.glob(os.path.join(pre.family_paths[f], "*.out"))
        for p in proteins:
            print(p)
            proteins_conf = []
            with open(p) as in_file:
                for line in in_file:
                    proteins_conf.append(line.strip())
            print(len(proteins_conf))
            test_size = int(0.25 * len(proteins_conf))
            val_size = int(0.15 * len(proteins_conf))
            train_all_p, test_p = train_test_split(proteins_conf, test_size=test_size, random_state=i)
            train_p, val_p = train_test_split(train_all_p, test_size = val_size, random_state=i)

            #preprocess
            train_p = pre.process_conf(train_p, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)
            val_p = pre.process_conf(val_p, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)
            test_p = pre.process_conf(test_p, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)
            print("train: " + repr(train_p.shape))
            print("val: " + repr(val_p.shape))
            print("test: " + repr(test_p.shape))
            del train_all_p
            del proteins_conf

            p_name = os.path.basename(p).split('.')[0]
            train_filename = os.path.join(path, f, "train", "train_"+ p_name +".npy")
            val_filename = os.path.join(path, f, "val", "val_"+ p_name +".npy")
            test_filename = os.path.join(path, f, "test", "test_"+ p_name +".npy")
            np.save(train_filename, train_p)
            np.save(val_filename, val_p)
            np.save(test_filename, test_p)

            del train_p
            del test_p
            del val_p
            
    # train autoencoders
    for f in pre.families:
        print("Training for family %s" %f)
        train = read_set_for_family(f,"train")
        test = read_set_for_family(f,"val")
        print("train: " + repr(train.shape))
        print("test" + repr(test.shape))
        ae = get_ae()
        ae.fit(train, train,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(test, test),
               callbacks=create_checkpoints(f),
               verbose=1)
        del train
        del test
    
    # evaluate autoencoders
    tp, tn, fp, fn = {}, {}, {}, {}
    tp_p, tn_p, fp_p, fn_p = {}, {}, {}, {}
    prec, recall, spec, auc = {}, {}, {}, {}
    prec_p, recall_p, spec_p, auc_p = {}, {}, {}, {}
    for f in pre.families:
        print("Evaluating family %s" %f)
        [tp[f], tn[f], fp[f], fn[f]], [tp_p[f], tn_p[f], fp_p[f], fn_p[f]] = evaluate_for_fam(f)
        prec_p[f] = (1.0* tp_p[f] / (tp_p[f] + fp_p[f]))
        recall_p[f] = (1.0* tp_p[f] / (tp_p[f] + fn_p[f]))
        spec_p[f] = (1.0* tn_p[f] / (tn_p[f] + fp_p[f]))
        auc_p[f] = (recall_p[f] + spec_p[f]) / 2
        # write to csv 
        print([i, f, tp_p[f], tn_p[f], fp_p[f], tn_p[f], prec_p[f], recall_p[f], spec_p[f], auc_p[f]])
        writer_fam.writerow([i, f, tp_p[f], tn_p[f], fp_p[f], tn_p[f], prec_p[f], recall_p[f], spec_p[f], auc_p[f]])
    prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p = 0, 0, 0, 0
    for f in pre.families:
        prec_wavg_p += lengths[f] * prec_p[f] / total
        recall_wavg_p += lengths[f] * recall_p[f] / total
        spec_wavg_p += lengths[f] * spec_p[f] / total
        auc_wavg_p += lengths[f] * auc_p[f] / total
    # write to csv
    print([i, prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p])
    writer_avg.writerow([i, prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p])
res_fam.close()
res_avg.close()

../dataset/families/fam_1/2EBN.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_1/1JFX.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_1/1ITX.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_1/1NAR.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_1/1KFW.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_1/1B1Y.lf_str.out
10000
(6000, 

(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_7/1OLL.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_8/1AF7.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_8/1Y8C.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_8/1DUS.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1500, 2004)
test: (2500, 2004)
../dataset/families/fam_8/1F3L.lf_str.out
10000
(6000, 668, 3)
(6000, 2004)
(1500, 668, 3)
(1500, 2004)
(2500, 668, 3)
(2500, 2004)
train: (6000, 2004)
val: (1

Epoch 3/20
48000/48000 [==============================] - 5s 108us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1930 - val_mean_absolute_error: 0.0518
Epoch 4/20
48000/48000 [==============================] - 5s 98us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1930 - val_mean_absolute_error: 0.0506
Epoch 5/20
48000/48000 [==============================] - 5s 98us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1930 - val_mean_absolute_error: 0.0529
Epoch 6/20
48000/48000 [==============================] - 5s 97us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1930 - val_mean_absolute_error: 0.0520
Epoch 7/20
48000/48000 [==============================] - 5s 96us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1930 - val_mean_absolute_error: 0.0520
Epoch 8/20
48000/48000 [==============================] - 5s 97us/step - loss: 0.1930 - mean_absolute_error: 0.0517 - val_loss: 0.1931 - val_mean_absolute

36000/36000 [==============================] - 4s 105us/step - loss: 0.1952 - mean_absolute_error: 0.0892 - val_loss: 0.1301 - val_mean_absolute_error: 0.0270
Epoch 2/20
36000/36000 [==============================] - 4s 98us/step - loss: 0.1254 - mean_absolute_error: 0.0223 - val_loss: 0.1227 - val_mean_absolute_error: 0.0191
Epoch 3/20
36000/36000 [==============================] - 4s 97us/step - loss: 0.1213 - mean_absolute_error: 0.0175 - val_loss: 0.1202 - val_mean_absolute_error: 0.0162
Epoch 4/20
36000/36000 [==============================] - 4s 98us/step - loss: 0.1196 - mean_absolute_error: 0.0154 - val_loss: 0.1191 - val_mean_absolute_error: 0.0148
Epoch 5/20
36000/36000 [==============================] - 4s 100us/step - loss: 0.1188 - mean_absolute_error: 0.0144 - val_loss: 0.1185 - val_mean_absolute_error: 0.0142
Epoch 6/20
36000/36000 [==============================] - 4s 99us/step - loss: 0.1183 - mean_absolute_error: 0.0140 - val_loss: 0.1181 - val_mean_absolute_error: 0.

Train on 30000 samples, validate on 7500 samples
Epoch 1/20
30000/30000 [==============================] - 3s 106us/step - loss: 0.1174 - mean_absolute_error: 0.0697 - val_loss: 0.0597 - val_mean_absolute_error: 0.0199
Epoch 2/20
30000/30000 [==============================] - 3s 99us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0597 - val_mean_absolute_error: 0.0198
Epoch 3/20
30000/30000 [==============================] - 3s 98us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0596 - val_mean_absolute_error: 0.0198
Epoch 4/20
30000/30000 [==============================] - 3s 98us/step - loss: 0.0596 - mean_absolute_error: 0.0198 - val_loss: 0.0596 - val_mean_absolute_error: 0.0198
Epoch 5/20
30000/30000 [==============================] - 3s 97us/step - loss: 0.0596 - mean_absolute_error: 0.0198 - val_loss: 0.0596 - val_mean_absolute_error: 0.0199
Epoch 6/20
30000/30000 [==============================] - 3s 97us/step - loss: 0.0596 - mean_absolute_err

...

36000/36000 [==============================] - 6s 166us/step - loss: 0.1239 - mean_absolute_error: 0.0206 - val_loss: 0.1216 - val_mean_absolute_error: 0.0179
Epoch 3/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1202 - mean_absolute_error: 0.0164 - val_loss: 0.1191 - val_mean_absolute_error: 0.0152
Epoch 4/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1181 - mean_absolute_error: 0.0140 - val_loss: 0.1173 - val_mean_absolute_error: 0.0130
Epoch 5/20
36000/36000 [==============================] - 6s 163us/step - loss: 0.1168 - mean_absolute_error: 0.0123 - val_loss: 0.1163 - val_mean_absolute_error: 0.0116
Epoch 6/20
36000/36000 [==============================] - 6s 160us/step - loss: 0.1159 - mean_absolute_error: 0.0111 - val_loss: 0.1156 - val_mean_absolute_error: 0.0107
Epoch 7/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1154 - mean_absolute_error: 0.0104 - val_loss: 0.1152 - val_mean_absolute_error

Train on 30000 samples, validate on 7500 samples
Epoch 1/20
30000/30000 [==============================] - 9s 298us/step - loss: 0.1172 - mean_absolute_error: 0.0696 - val_loss: 0.0598 - val_mean_absolute_error: 0.0199
Epoch 2/20
30000/30000 [==============================] - 5s 162us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0596 - val_mean_absolute_error: 0.0198
Epoch 3/20
30000/30000 [==============================] - 5s 162us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0597 - val_mean_absolute_error: 0.0198
Epoch 4/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0596 - val_mean_absolute_error: 0.0198
Epoch 5/20
30000/30000 [==============================] - 5s 164us/step - loss: 0.0597 - mean_absolute_error: 0.0198 - val_loss: 0.0597 - val_mean_absolute_error: 0.0198
Epoch 6/20
30000/30000 [==============================] - 5s 167us/step - loss: 0.0596 - mean_absolut

Train on 30000 samples, validate on 7500 samples
Epoch 1/20
30000/30000 [==============================] - 9s 312us/step - loss: 0.2954 - mean_absolute_error: 0.1109 - val_loss: 0.2596 - val_mean_absolute_error: 0.0784
Epoch 2/20
30000/30000 [==============================] - 5s 181us/step - loss: 0.2595 - mean_absolute_error: 0.0785 - val_loss: 0.2595 - val_mean_absolute_error: 0.0787
Epoch 3/20
30000/30000 [==============================] - 5s 181us/step - loss: 0.2595 - mean_absolute_error: 0.0785 - val_loss: 0.2595 - val_mean_absolute_error: 0.0788
Epoch 4/20
30000/30000 [==============================] - 5s 169us/step - loss: 0.2595 - mean_absolute_error: 0.0785 - val_loss: 0.2594 - val_mean_absolute_error: 0.0784
Epoch 5/20
30000/30000 [==============================] - 5s 177us/step - loss: 0.2595 - mean_absolute_error: 0.0785 - val_loss: 0.2595 - val_mean_absolute_error: 0.0785
Epoch 6/20
30000/30000 [==============================] - 5s 171us/step - loss: 0.2595 - mean_absolut

Max loss is 0.064131
Test for fam fam_1
Test for fam fam_2
Test for fam fam_3
Test for fam fam_4
Test for fam fam_5
Test for fam fam_6
Test for fam fam_7
Test for fam fam_8
Test for fam fam_9
[19, 'fam_1', 9, 43, 3, 43, 0.75, 1.0, 0.9347826086956522, 0.9673913043478262]
Evaluating family fam_2
Test for autoencoder on fam fam_2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_173 (InputLayer)       (None, 2004)              0         
_________________________________________________________________
dense_345 (Dense)            (None, 25)                50125     
_________________________________________________________________
dense_346 (Dense)            (None, 2004)              52104     
Total params: 102,229
Trainable params: 102,229
Non-trainable params: 0
_________________________________________________________________
Max loss is 0.039531
Test for fam fam_1
Test for fam fam_2
Test for fam

Max loss is 0.042697
Test for fam fam_1
Test for fam fam_2
Test for fam fam_3
Test for fam fam_4
Test for fam fam_5
Test for fam fam_6
Test for fam fam_7
Test for fam fam_8
Test for fam fam_9
[19, 'fam_9', 5, 50, 0, 50, 1.0, 1.0, 1.0, 1.0]
[19, 0.7360910694244027, 1.0000000000000002, 0.9386888767224435, 0.9693444383612217]
